# [SQL Interview Questions on Data Lemur - Easy](https://datalemur.com/questions?difficulty=Easy&category=SQL)

##### Solved by: Dorothy Kunth

### 1. [Histogram of Tweets - Twitter](https://datalemur.com/questions/sql-histogram-tweets)

Assume you're given a table Twitter tweet data, write a query to obtain a histogram of tweets posted per user in 2022. Output the tweet count per user as the bucket and the number of Twitter users who fall into that bucket.

In other words, group the users by the number of tweets they posted in 2022 and count the number of users in each group.

``tweets`` table

| tweet_id | user_id | msg                                                               | tweet_date          |
|----------|---------|-------------------------------------------------------------------|---------------------|
| 241425   | 254     | If the salary is so competitive why won’t you tell me what it is? | 03/01/2022 11:00:00 |
| 214252   | 111     | Am considering taking Tesla private at $420. Funding secured.     | 12/30/2021 00:00:00 |
| 739252   | 111     | Despite the constant negative press covfefe                       | 01/01/2022 11:00:00 |
| 846402   | 111     | Following @NickSinghTech on Twitter changed my life!              | 02/14/2022 11:00:00 |
| 231574   | 148     | I no longer have a manager. I can't be managed                    | 03/23/2022 11:00:00 |


### Solution
1. Query the number of tweets per user in 2022

```sql
SELECT
    user_id,
    COUNT(tweet_id) AS tweet_count
FROM tweets
WHERE tweet_date BETWEEN '01/01/2022' AND '12/31/2022'
GROUP BY 1
```
| user_id | tweet_count |
|---------|-------------|
| 111     | 2           |
| 148     | 1           |
| 254     | 1           |


<br><br>
2. Use the above query as an inline query (FROM clause) and use the ``tweet_count`` as bucket then count the number of users per bucket

```sql
SELECT 
  tweet_count AS bucket,
  COUNT(user_id) num_users
FROM (
  SELECT 
   user_id,
   COUNT(tweet_id) AS tweet_count
  FROM tweets
  WHERE tweet_date BETWEEN '01/01/2022' AND '12/31/2022'
  GROUP BY 1) AS tweet_count_per_user
GROUP BY 1
```
| bucket | num_users |
|--------|-----------|
| 1      | 2         |
| 2      | 1         |


### 2. [Data Science Skills - LinkedIn](https://datalemur.com/questions/matching-skills)

Given a table of candidates and their skills, you're tasked with finding the candidates best suited for an open Data Science job. You want to find candidates who are proficient in Python, Tableau, and PostgreSQL.

Write a query to list the candidates who possess all of the required skills for the job. Sort the output by candidate ID in ascending order.

Assumption: There are no duplicates in the candidates table.

``candidates`` table

| candidate_id | skill      |
|--------------|------------|
| 123          | Python     |
| 123          | Tableau    |
| 123          | PostgreSQL |
| 234          | R          |
| 234          | PowerBI    |
| 234          | SQL Server |
| 345          | Python     |
| 345          | Tableau    |
| 147          | Python     |
| 147          | PostgreSQL |
| 147          | Tableau    |
| 147          | Java       |
| 168          | Python     |
| 256          | Tableau    |


### Solution
1. Select all the candidates that have skills in Python, Tableau, and PostgreSQL

```sql
SELECT *
FROM candidates
WHERE skill IN ('Python', 'Tableau', 'PostgreSQL')
ORDER BY candidate_id
```
| candidate_id | skill      |
|--------------|------------|
| 123          | Python     |
| 123          | Tableau    |
| 123          | PostgreSQL |
| 147          | Python     |
| 147          | PostgreSQL |
| 147          | Tableau    |
| 168          | Python     |
| 256          | Tableau    |
| 345          | Tableau    |
| 345          | Python     |

<br><br>
2. Use the above query as an inline query (FROM clause), group by ``candidate_id`` then use HAVING clause to filter only the count of ``skill`` to 3

```sql
SELECT candidate_id
FROM (SELECT *
      FROM candidates
      WHERE skill IN ('Python', 'Tableau', 'PostgreSQL')
      ORDER BY candidate_id) AS skillset
GROUP BY candidate_id
HAVING COUNT(skill) = 3
ORDER BY candidate_id;
```
| candidate_id |
|--------------|
| 123          |
| 147          |


### 3. [Page With No Likes - Facebook](https://datalemur.com/questions/sql-page-with-no-likes)

Assume you're given two tables containing data about Facebook Pages and their respective likes (as in "Like a Facebook Page").

Write a query to return the IDs of the Facebook pages that have zero likes. The output should be sorted in ascending order based on the page IDs.

``pages`` table

| page_id | page_name              |
|---------|------------------------|
| 20001   | SQL Solutions          |
| 20045   | Brain Exercises        |
| 20701   | Tips for Data Analysts |
| 31111   | Postgres Crash Course  |
| 32728   | Break the thread       |


``page_likes`` table

| user_id | page_id | liked_date          |
|---------|---------|---------------------|
| 111     | 20001   | 04/08/2022 00:00:00 |
| 121     | 20045   | 03/12/2022 00:00:00 |
| 156     | 20001   | 07/25/2022 00:00:00 |
| 255     | 20045   | 07/19/2022 00:00:00 |
| 125     | 20001   | 07/19/2022 00:00:00 |
| 144     | 31111   | 06/21/2022 00:00:00 |
| 125     | 31111   | 07/04/2022 00:00:00 |


### Solution

```sql
SELECT p.page_id
FROM pages AS p
LEFT JOIN page_likes AS pl 
ON p.page_id = pl.page_id
WHERE liked_date IS NULL
ORDER BY p.page_id
```
| page_id |
|---------|
| 20701   |
| 32728   |


### 4. [Unfinished Parts - Tesla](https://datalemur.com/questions/tesla-unfinished-parts)

Tesla is investigating production bottlenecks and they need your help to extract the relevant data. Write a query to determine which parts have begun the assembly process but are not yet finished.

Assumptions:

- parts_assembly table contains all parts currently in production, each at varying stages of the assembly process.
- An unfinished part is one that lacks a finish_date.

``parts_assembly`` table

| part    | finish_date         | assembly_step |
|---------|---------------------|---------------|
| battery | 01/22/2022 00:00:00 | 1             |
| battery | 02/22/2022 00:00:00 | 2             |
| battery | 03/22/2022 00:00:00 | 3             |
| bumper  | 01/22/2022 00:00:00 | 1             |
| bumper  | 02/22/2022 00:00:00 | 2             |
| bumper  | NULL                | 3             |
| bumper  | NULL                | 4             |
| door    | 01/22/2022 00:00:00 | 1             |
| door    | 02/22/2022 00:00:00 | 2             |
| engine  | 01/01/2022 00:00:00 | 1             |
| engine  | 01/25/2022 00:00:00 | 2             |
| engine  | 02/28/2022 00:00:00 | 3             |
| engine  | 04/01/2022 00:00:00 | 4             |
| engine  | NULL                | 5             |


### Solution

```sql
SELECT part, assembly_step
FROM parts_assembly
WHERE finish_date IS NULL;
```
| part   | assembly_step |
|--------|---------------|
| bumper | 3             |
| bumper | 4             |
| engine | 5             |


### 5. [Laptop vs. Mobile Viewership - The New York Times](https://datalemur.com/questions/laptop-mobile-viewership)

Assume you're given the table on user viewership categorised by device type where the three types are laptop, tablet, and phone.

Write a query that calculates the total viewership for laptops and mobile devices where mobile is defined as the sum of tablet and phone viewership. Output the total viewership for laptops as laptop_views and the total viewership for mobile devices as mobile_views.

``viewership`` table

| user_id | device_type | view_time           | lap_mob | device_family | laptop_views | mobile_views | mobile |
|---------|-------------|---------------------|---------|---------------|--------------|--------------|--------|
| 123     | tablet      | 01/02/2022 00:00:00 | NULL    | NULL          | NULL         | NULL         | NULL   |
| 125     | laptop      | 01/07/2022 00:00:00 | NULL    | NULL          | NULL         | NULL         | NULL   |
| 128     | laptop      | 02/09/2022 00:00:00 | NULL    | NULL          | NULL         | NULL         | NULL   |
| 129     | phone       | 02/09/2022 00:00:00 | NULL    | NULL          | NULL         | NULL         | NULL   |
| 145     | tablet      | 02/24/2022 00:00:00 | NULL    | NULL          | NULL         | NULL         | NULL   |


### Solution 1 
1. Create a case statement

```sql
SELECT 
  CASE WHEN device_type = 'laptop' THEN 1 ELSE 0 END AS laptop_views,
  CASE WHEN device_type IN ('phone', 'tablet') THEN 1 ELSE 0 END AS mobile_views
FROM viewership
```
| laptop_views | mobile_views |   |   |   |   |   |   |
|--------------|--------------|---|---|---|---|---|---|
| 0            | 1            |   |   |   |   |   |   |
| 1            | 0            |   |   |   |   |   |   |
| 1            | 0            |   |   |   |   |   |   |
| 0            | 1            |   |   |   |   |   |   |
| 0            | 1            |   |   |   |   |   |   |

<br><br>
2. Use the above query as an inline query (FROM clause) and sum both views

```sql
SELECT SUM(laptop_views) AS laptop_views, SUM(mobile_views) AS mobile_views
FROM (
  SELECT 
    CASE WHEN device_type = 'laptop' THEN 1 ELSE 0 END AS laptop_views,
    CASE WHEN device_type IN ('phone', 'tablet') THEN 1 ELSE 0 END AS mobile_views
  FROM viewership) AS views
```
| laptop_views | mobile_views |
|--------------|--------------|
| 2            | 3            |


### Solution 2
```sql
SELECT 
    SUM(CASE WHEN device_type = 'laptop' THEN 1 ELSE 0 END) AS laptop_views, 
    SUM(CASE WHEN device_type IN ('phone', 'tablet') THEN 1 ELSE 0 END) AS mobile_views
FROM viewership
```
| laptop_views | mobile_views |
|--------------|--------------|
| 2            | 3            |


### 6. [Average Post Hiatus - Facebook](https://datalemur.com/questions/sql-average-post-hiatus-1)

Given a table of Facebook posts, for each user who posted at least twice in 2021, write a query to find the number of days between each user’s first post of the year and last post of the year in the year 2021. Output the user and number of the days between each user's first and last post.

``posts`` table

| user_id | post_id | post_content                                                                                                                            | post_date           | difference |
|---------|---------|-----------------------------------------------------------------------------------------------------------------------------------------|---------------------|------------|
| 151652  | 111766  | it's always winter, but never Christmas.                                                                                                | 12/01/2021 11:00:00 | NULL       |
| 661093  | 442560  | Bed. Class 8-12. Work 12-3. Gym 3-5 or 6. Then class 6-10. Another day that's gonna fly by. I miss my girlfriend                        | 09/08/2021 10:00:00 | NULL       |
| 661093  | 624356  | Happy valentines!                                                                                                                       | 02/14/2021 00:00:00 | NULL       |
| 151652  | 599415  | Need a hug                                                                                                                              | 01/28/2021 00:00:00 | NULL       |
| 178425  | 157336  | I'm so done with these restrictions - I want to travel!!!                                                                               | 03/24/2021 11:00:00 | NULL       |
| 423967  | 784254  | Just going to cry myself to sleep after watching Marley and Me.                                                                         | 05/05/2021 00:00:00 | NULL       |
| 151325  | 613451  | Happy new year all my friends!                                                                                                          | 01/01/2022 11:00:00 | NULL       |
| 151325  | 987562  | The global surface temperature for June 2022 was the sixth-highest in the 143-year record. This is definitely global warming happening. | 07/01/2022 10:00:00 | NULL       |
| 661093  | 674356  | Can't wait to start my freshman year - super excited!                                                                                   | 08/18/2021 10:00:00 | NULL       |
| 151325  | 451464  | Garage sale this Saturday 1 PM. All welcome to check out!                                                                               | 10/25/2021 10:00:00 | NULL       |
| 151652  | 994156  | Does anyone have an extra iPhone charger to sell?                                                                                       | 04/01/2021 10:00:00 | NULL       |


### Solution 1 
Cast the ``post_date`` to DATE then get the difference between the MAX and MIN dates. Then filter to year 2021 and where number of ``post_id`` is > 1

```sql
SELECT 
	user_id, 
    MAX(CAST(post_date AS DATE)) - MIN(CAST(post_date AS DATE)) AS days_between
FROM posts
WHERE DATE_PART('year', post_date) = 2021 
GROUP BY user_id
HAVING COUNT(post_id) > 1;
```
| user_id | days_between |
|---------|--------------|
| 151652  | 307          |
| 661093  | 206          |


### Solution 2

1. Use window function LEAD on ``post_date`` to get the subsequent row as ``next_post_date``. Then filter to year 2021

```sql
SELECT 
  user_id, 
  post_id, 
  post_date, 
  LEAD(post_date) OVER (PARTITION BY user_id ORDER BY post_date) AS next_post_date
FROM posts
WHERE DATE_PART('year', post_date) = 2021
ORDER BY 1, 3
```
| user_id | post_id | post_date           | next_post_date      |
|---------|---------|---------------------|---------------------|
| 151325  | 451464  | 10/25/2021 10:00:00 | NULL                |
| 151652  | 599415  | 01/28/2021 00:00:00 | 04/01/2021 10:00:00 |
| 151652  | 994156  | 04/01/2021 10:00:00 | 12/01/2021 11:00:00 |
| 151652  | 111766  | 12/01/2021 11:00:00 | NULL                |
| 178425  | 157336  | 03/24/2021 11:00:00 | NULL                |
| 423967  | 784254  | 05/05/2021 00:00:00 | NULL                |
| 661093  | 624356  | 02/14/2021 00:00:00 | 08/18/2021 10:00:00 |
| 661093  | 674356  | 08/18/2021 10:00:00 | 09/08/2021 10:00:00 |
| 661093  | 442560  | 09/08/2021 10:00:00 | NULL                |



2. Use the above query as an inline query (FROM clause) and sum the days between ``next_post_date`` and ``post_date``. Then filter where ``next_post_date`` is not null.

```sql
SELECT
user_id,
SUM(DATE_PART('day', next_post_date - post_date)) AS days_between
FROM (
  SELECT 
    user_id, 
    post_id, 
    post_date, 
    LEAD(post_date) OVER (PARTITION BY user_id ORDER BY post_date) AS next_post_date
  FROM posts
  WHERE DATE_PART('year', post_date) = 2021
  ORDER BY 1, 3) AS user_post_2021
WHERE next_post_date IS NOT NULL
GROUP BY 1
```
| user_id | days_between |
|---------|--------------|
| 151652  | 307          |
| 661093  | 206          |


### 7. [Teams Power Users - Microsoft](https://datalemur.com/questions/teams-power-users)
Write a query to identify the top 2 Power Users who sent the highest number of messages on Microsoft Teams in August 2022. Display the IDs of these 2 users along with the total number of messages they sent. Output the results in descending order based on the count of the messages.

Assumption:

No two users have sent the same number of messages in August 2022.

``messages`` table

| message_id | sender_id | receiver_id | content                                       | sent_date           | message_count |
|------------|-----------|-------------|-----------------------------------------------|---------------------|---------------|
| 901        | 3601      | 4500        | You up?                                       | 08/03/2022 16:43:00 | NULL          |
| 743        | 3601      | 8752        | Let's take this offline                       | 06/14/2022 14:30:00 | NULL          |
| 888        | 3601      | 7855        | DataLemur has awesome user base!              | 08/12/2022 08:45:00 | NULL          |
| 100        | 2520      | 6987        | Send this out now!                            | 08/16/2021 00:35:00 | NULL          |
| 898        | 2520      | 9630        | Are you ready for your upcoming presentation? | 08/13/2022 14:35:00 | NULL          |
| 990        | 2520      | 8520        | Maybe it was done by the automation process.  | 08/19/2022 06:30:00 | NULL          |
| 819        | 2310      | 4500        | What's the status on this?                    | 07/10/2022 15:55:00 | NULL          |
| 922        | 3601      | 4500        | Get on the call                               | 08/10/2022 17:03:00 | NULL          |
| 942        | 2520      | 3561        | How much do you know about Data Science?      | 08/17/2022 13:44:00 | NULL          |
| 966        | 3601      | 7852        | Meet me in five!                              | 08/17/2022 02:20:00 | NULL          |
| 902        | 4500      | 3601        | Only if you're buying                         | 08/03/2022 06:50:00 | NULL          |

### Solution

```sql
SELECT 
  sender_id, 
  COUNT(message_id) AS message_count
FROM messages
WHERE EXTRACT(MONTH FROM sent_date) = 8
  AND EXTRACT(YEAR FROM sent_date) = 2022
GROUP BY sender_id
ORDER BY COUNT(message_id) DESC
LIMIT 2
```
| sender_id | message_count |
|-----------|---------------|
| 3601      | 4             |
| 2520      | 3             |

### 8. [Duplicate Job Listings - LinkedIn](https://datalemur.com/questions/duplicate-job-listings)
Assume you're given a table containing job postings from various companies on the LinkedIn platform. Write a query to retrieve the count of companies that have posted duplicate job listings.

Definition:

Duplicate job listings are defined as two job listings within the same company that share identical titles and descriptions.

``job_listings`` table

| company_id | title                         | job_id | description                                                                                                                                                                              |
|------------|-------------------------------|--------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 827        | Business Analyst              | 248    | Business analyst evaluates past and current business data with the primary goal of improving decision-making processes within organizations.                                             |
| 845        | Business Analyst              | 149    | Business analyst evaluates past and current business data with the primary goal of improving decision-making processes within organizations.                                             |
| 345        | Data Analyst                  | 945    | Data analyst reviews data to identify key insights into a business's customers and ways the data can be used to solve problems.                                                          |
| 345        | Data Analyst                  | 164    | Data analyst reviews data to identify key insights into a business's customers and ways the data can be used to solve problems.                                                          |
| 244        | Data Engineer                 | 172    | Data engineer works in a variety of settings to build systems that collect, manage, and convert raw data into usable information for data scientists and business analysts to interpret. |
| 827        | Data Scientist                | 256    | Data scientist uses data to understand and explain the phenomena around them, and help organizations make better decisions.                                                              |
| 244        | Software Engineer             | 365    | Software engineers design and create computer systems and applications to solve real-world problems.                                                                                     |
| 400        | Business Intelligence Analyst | 674    | Business intelligence analyst reviews data to produce finance and market intelligence reports.                                                                                           |
| 827        | Data Scientist                | 245    | Data scientist uses data to understand and explain the phenomena around them, and help organizations make better decisions.                                                              |
| 244        | Software Engineer             | 301    | Software engineers design and create computer systems and applications to solve real-world problems.                                                                                     |

### Solution
1. Since job_id is unique even if the title and description are identical, count the number of job_id grouped by company_id, title, and description

```sql
SELECT 
  company_id, 
  title, 
  description, 
  COUNT(job_id) as job_count
FROM job_listings
GROUP BY company_id, title, description
```
| company_id | title                         | description                                                                                                                                                                              | job_count |
|------------|-------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------|
| 827        | Data Scientist                | Data scientist uses data to understand and explain the phenomena around them, and help organizations make better decisions.                                                              | 2         |
| 244        | Data Engineer                 | Data engineer works in a variety of settings to build systems that collect, manage, and convert raw data into usable information for data scientists and business analysts to interpret. | 1         |
| 845        | Business Analyst              | Business analyst evaluates past and current business data with the primary goal of improving decision-making processes within organizations.                                             | 1         |
| 244        | Software Engineer             | Software engineers design and create computer systems and applications to solve real-world problems.                                                                                     | 2         |
| 345        | Data Analyst                  | Data analyst reviews data to identify key insights into a business's customers and ways the data can be used to solve problems.                                                          | 2         |
| 827        | Business Analyst              | Business analyst evaluates past and current business data with the primary goal of improving decision-making processes within organizations.                                             | 1         |
| 400        | Business Intelligence Analyst | Business intelligence analyst reviews data to produce finance and market intelligence reports.      | 1         |

<br><br>
2. Use the above query as an inline query (FROM clause) then count the number of unique companies where job_count is > 1
```sql
SELECT COUNT(DISTINCT company_id) num_companies_with_dup
FROM (
  SELECT 
    company_id, 
    title, 
    description, 
  COUNT(job_id) as job_count
  FROM job_listings
  GROUP BY company_id, title, description) grouped
WHERE job_count > 1
```
| num_companies_with_dup |
|------------------------|
| 3                      |


### 9. [Cities With Completed Trades - Robinhood Markets](https://datalemur.com/questions/completed-trades)
Assume you're given the tables containing completed trade orders and user details in a Robinhood trading system.

Write a query to retrieve the top three cities that have the highest number of completed trade orders listed in descending order. Output the city name and the corresponding number of completed trade orders.

``trades`` table


| order_id | user_id | quantity | status    | date                | price |
|----------|---------|----------|-----------|---------------------|-------|
| 100101   | 111     | 10       | Cancelled | 08/17/2022 12:00:00 | 9.80  |
| 100102   | 111     | 10       | Completed | 08/17/2022 12:00:00 | 10.00 |
| 100264   | 148     | 40       | Completed | 08/26/2022 12:00:00 | 4.80  |
| 100305   | 300     | 15       | Completed | 09/05/2022 12:00:00 | 10.00 |
| 100909   | 488     | 1        | Completed | 07/05/2022 12:00:00 | 6.50  |
| 100259   | 148     | 35       | Completed | 08/25/2022 12:00:00 | 5.10  |
| 100900   | 148     | 50       | Completed | 07/14/2022 12:00:00 | 9.78  |
| 101432   | 265     | 10       | Completed | 08/16/2022 12:00:00 | 13.00 |
| 102533   | 488     | 25       | Cancelled | 11/10/2022 12:00:00 | 22.40 |
| 100565   | 265     | 2        | Completed | 09/27/2022 12:00:00 | 8.70  |
| 100400   | 178     | 32       | Completed | 09/17/2022 12:00:00 | 12.00 |
| 100777   | 178     | 60       | Completed | 07/25/2022 17:47:00 | 3.56  |

``users`` table

| user_id | city          | email                         | signup_date         |
|---------|---------------|-------------------------------|---------------------|
| 111     | San Francisco | rrok10@gmail.com              | 08/03/2021 12:00:00 |
| 148     | Boston        | sailor9820@gmail.com          | 08/20/2021 12:00:00 |
| 178     | San Francisco | harrypotterfan182@gmail.com   | 01/05/2022 12:00:00 |
| 265     | Denver        | shadower_@hotmail.com         | 02/26/2022 12:00:00 |
| 300     | San Francisco | houstoncowboy1122@hotmail.com | 06/30/2022 12:00:00 |
| 488     | New York      | empire_state78@outlook.com    | 07/03/2022 12:00:00 |

### Solution

```sql
SELECT
  city,
  COUNT(*) AS total_trade_orders
FROM
  trades AS t
LEFT JOIN 
  users AS u
ON t.user_id = u.user_id
WHERE status = 'Completed'
GROUP BY city
ORDER BY total_trade_orders DESC
LIMIT 3
```

| city          | total_trade_orders |
|---------------|--------------------|
| San Francisco | 4                  |
| Boston        | 3                  |
| Denver        | 2                  |

### 10. [Average Review Ratings - Amazon](https://datalemur.com/questions/sql-avg-review-ratings)
Given the reviews table, write a query to retrieve the average star rating for each product, grouped by month. The output should display the month as a numerical value, product ID, and average star rating rounded to two decimal places. Sort the output first by month and then by product ID.

``reviews table``

| review_id | user_id | submit_date         | product_id | stars |
|-----------|---------|---------------------|------------|-------|
| 6171      | 123     | 06/08/2022 00:00:00 | 50001      | 4     |
| 7802      | 265     | 06/10/2022 00:00:00 | 69852      | 4     |
| 5293      | 362     | 06/18/2022 00:00:00 | 50001      | 3     |
| 6352      | 192     | 07/26/2022 00:00:00 | 69852      | 3     |
| 4517      | 981     | 07/05/2022 00:00:00 | 69852      | 2     |
| 2501      | 142     | 06/21/2022 00:00:00 | 12580      | 5     |
| 4582      | 562     | 06/15/2022 00:00:00 | 12580      | 4     |
| 2536      | 136     | 07/04/2022 00:00:00 | 11223      | 5     |
| 1200      | 100     | 05/17/2022 00:00:00 | 25255      | 4     |
| 2555      | 232     | 05/31/2022 00:00:00 | 25600      | 4     |
| 2556      | 167     | 05/31/2022 00:00:00 | 25600      | 5     |
| 1301      | 120     | 05/18/2022 00:00:00 | 25600      | 4     |

### Solution

```sql
SELECT
  DATE_PART('month', CAST(submit_date AS DATE)) AS month,
  product_id,
  ROUND(AVG(stars), 2) AS average_stars
FROM reviews
GROUP BY month, product_id
ORDER BY month, product_id
```

| month | product_id | average_stars |
|-------|------------|---------------|
| 5     | 25255      | 4.00          |
| 5     | 25600      | 4.33          |
| 6     | 12580      | 4.50          |
| 6     | 50001      | 3.50          |
| 6     | 69852      | 4.00          |
| 7     | 11223      | 5.00          |
| 7     | 69852      | 2.50          |


##### in progress...